In [119]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import re
import wikipediaapi as wiki
from datetime import datetime
import random
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import wikipedia
import time
import urllib
import urllib.request
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException

import time

In [ ]:
##########################
###     CONSTANTS      ###
##########################

In [96]:
# IMdb API key
api_key = "k_11m5rq35"

# headers
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive',
        "referer":"referer: https://www.google.com/",
        "user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36"
        }

cookies = {"CONSENT":"YES+shp.gws-20210330-0-RC1.de+FX+412"}

# options = Options()
# options.headless = True

# Parent and Relative strings to search for in wikipedia infobox
parent_pattern = re.compile('Parent')
relative_pattern = re.compile('Relative')
starring_pattern = re.compile('Starring')
voice_pattern = re.compile('Voices of')
wikipedia_pattern = re.compile("wiki")

not_nepo_relationships = 'son|daughter|nephew|niece|grandson|grand-daughter|grandnephew|grandniece|in-law|stepson|stepdaughter|brother|sister|cousin'

# url base
wiki_url_base = "https://en.wikipedia.org" # wikipedia
imdb_base_url = 'https://www.imdb.com/title/'

# wikipedia language setting
wiki_wiki = wiki.Wikipedia('en')


# ia = imdb.Cinemagoer()

In [146]:
top_250_mov_url = "https://www.imdb.com/chart/top/?ref_=nv_mv_250"
top_250_tv_url = "https://www.imdb.com/chart/toptv/?ref_=nv_tvv_250"
pop_mov_url = "https://www.imdb.com/search/title/?title_type=feature"
pop_tv_url = "https://www.imdb.com/search/title/?title_type=tv_series"

In [141]:
# [ input  ] imdb url: string, type: string, class_name: string (class of list item on imdb list page)
# [ output ] dataframe
# [  goal  ] gets title, link, image, year for each title in an imdb list
def get_top_lists(url, type, class_name):

    options = Options()
    options.headless = False

    service = Service('/Users/jessicacarr/documents/j/github/are_they_a_nepo/chromedriver')
    driver = webdriver.Chrome(options=options, service=service)
    driver.get(url)

    if('search' in url):
          click_button_four_times(driver)
          print("after clicking")

    # class_name = "ipc-metadata-list-summary-item.sc-10233bc-0.iherUv.cli-parent"    
    elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, class_name))
     )
    
    print(f"Number of elements found: {len(elements)}")

    columns = ["image", "link", "title", "year", "type"]
    df = pd.DataFrame(columns=columns)

    for i, element in enumerate(elements, 1):

            try:
                img_element = element.find_element(By.CLASS_NAME, "ipc-image")
                img_src = img_element.get_attribute('src')
            except NoSuchElementException:
                img_src = ""  # Set img_src to empty string if element is not found

            a_element = element.find_element(By.CLASS_NAME, "ipc-title-link-wrapper")
            a_href = a_element.get_attribute('href')

            title_element = element.find_element(By.CLASS_NAME, "ipc-title__text")
            title = title_element.text
            title = title.split(". ",1)[1]
            print(title)

            try:
                year_element = element.find_element(By.CSS_SELECTOR, ".sc-b189961a-8.kLaxqf.cli-title-metadata-item, .sc-b189961a-8.kLaxqf.dli-title-metadata-item")
                year = year_element.text
            except NoSuchElementException:
                year = ""


            title = {
                "image" : img_src,
                "link": a_href,
                "title": title,
                "year": year,
                "type": type

            }    

            temp_df = pd.DataFrame([title])
            df = pd.concat([df, temp_df], ignore_index=True)

    driver.quit()

    return df
    

    

In [140]:
# [ input  ] driver: chromedriver
# [ output ] 
# [  goal  ] clicks on a button on an imdb page 9 times
def click_button_four_times(driver):
    click_count = 0 
    class_name = "ipc-btn.ipc-btn--single-padding.ipc-btn--center-align-content.ipc-btn--default-height.ipc-btn--core-base.ipc-btn--theme-base.ipc-btn--on-accent2.ipc-text-button.ipc-see-more__button"

    while click_count < 5:  # Loop until 9 clicks have been made
        # Wait until the button is present in the DOM and is clickable
        button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, class_name))
        )
        # Scroll the button into view
        driver.execute_script("arguments[0].scrollIntoView(true);", button)
        # Wait until the button is clickable
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, class_name))
        )
        # Try clicking the button
        try:
            button.click()
        except ElementClickInterceptedException:
            # If element is not clickable, try JavaScript click
            driver.execute_script("arguments[0].click();", button)
        click_count += 1  # Increment the counter after each click


In [90]:
top_250_mov = get_top_lists(top_250_mov_url, "film", "ipc-metadata-list-summary-item.sc-10233bc-0.iherUv.cli-parent")
top_250_tv = get_top_lists(top_250_tv_url, "tv", "ipc-metadata-list-summary-item.sc-10233bc-0.iherUv.cli-parent")

/var/folders/fb/pmmnfq35613dv6n8fynzxt8w0000gn/T/ipykernel_91199/464806522.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(title, ignore_index=True)
/var/folders/fb/pmmnfq35613dv6n8fynzxt8w0000gn/T/ipykernel_91199/464806522.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(title, ignore_index=True)
/var/folders/fb/pmmnfq35613dv6n8fynzxt8w0000gn/T/ipykernel_91199/464806522.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(title, ignore_index=True)
/var/folders/fb/pmmnfq35613dv6n8fynzxt8w0000gn/T/ipykernel_91199/464806522.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.a

In [145]:
pop_250_mov = get_top_lists(pop_mov_url, "film", "sc-74bf520e-3.klvfeN.dli-parent")
pop_250_tv = get_top_lists(pop_tv_url, "tv", "sc-74bf520e-3.klvfeN.dli-parent")

after clicking
Number of elements found: 250
Inside Out 2
Furiosa: A Mad Max Saga
Nosferatu
A Quiet Place: Day One
The Bikeriders
Trigger Warning
Horizon: An American Saga - Chapter 1
Hit Man
Here
Kinds of Kindness
IF
Inside Out
Bad Boys: Ride or Die
A Family Affair
Deadpool & Wolverine
Red One
Kalki 2898 AD
Longlegs
Oppenheimer
The Watchers
MaXXXine
The Fall Guy
The Ministry of Ungentlemanly Warfare
Mad Max: Fury Road
Heretic
Dune: Part Two
Civil War
Despicable Me 4
Anyone But You
Kingdom of the Planet of the Apes
Thelma
Superman
Under Paris
A Quiet Place
Twisters
X
Munjya
Ghostbusters: Frozen Empire
Challengers
In a Violent Nature
Poor Things
Maharaj
Beverly Hills Cop: Axel F
Monkey Man
Godzilla Minus One
The Exorcism
I Saw the TV Glow
Godzilla x Kong: The New Empire
Kiss the Girls
Madame Web
The Boy and the Heron
The First Omen
Kung Fu Panda 4
Abigail
The Idea of You
Presumed Innocent
Dune
A Quiet Place Part II
The Shawshank Redemption
Pearl
Fly Me to the Moon
Midsommar
The End We S

TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x0000000109c870e8 chromedriver + 5169384
1   chromedriver                        0x0000000109c7efba chromedriver + 5136314
2   chromedriver                        0x00000001097fb36c chromedriver + 402284
3   chromedriver                        0x0000000109848740 chromedriver + 718656
4   chromedriver                        0x0000000109848a01 chromedriver + 719361
5   chromedriver                        0x000000010988dbc4 chromedriver + 1002436
6   chromedriver                        0x000000010986badd chromedriver + 862941
7   chromedriver                        0x000000010988af57 chromedriver + 991063
8   chromedriver                        0x000000010986b853 chromedriver + 862291
9   chromedriver                        0x000000010983b5c6 chromedriver + 665030
10  chromedriver                        0x000000010983be4e chromedriver + 667214
11  chromedriver                        0x0000000109c49d00 chromed

In [147]:
pop_250_tv = get_top_lists(pop_tv_url, "tv", "sc-74bf520e-3.klvfeN.dli-parent")

after clicking
Number of elements found: 250
The Boys
House of the Dragon
The Bear
The Acolyte
Bridgerton
Game of Thrones
Your Honor
Dexter
My Lady Jane
Doctor Who
Criminal Minds
Fallout
Breaking Bad
Suits
Stranger Things
Yellowstone
Grey's Anatomy
Mayor of Kingstown
The Sopranos
Demon Slayer: Kimetsu no Yaiba
The Walking Dead
Supernatural
Doctor Who
The Rookie
Shōgun
Rick and Morty
Young Sheldon
Gen V
Law & Order: Special Victims Unit
True Detective
NCIS
Supacell
House
Shameless
The Office
Evil
Sweet Tooth
Smiling Friends
From
The Big Bang Theory
The Gentlemen
Better Call Saul
Attack on Titan
Psych
The Last of Us
That '90s Show
Outlander
3 Body Problem
Modern Family
Interview with the Vampire
Peaky Blinders
Friends
How I Met Your Mother
Severance
9-1-1
Orphan Black: Echoes
Midsomer Murders
Sex Education
Hacks
Succession
Black Mirror
Vikings
One Piece
Ted Lasso
Monk
Fargo
Lost
American Horror Story
Bones
Euphoria
White Collar
The Chosen
Brooklyn Nine-Nine
Gangs of Galicia
Shoresy
The C

In [150]:
pop_250_tv.head(10)

,image,link,title,year,type
0,https://m.media-amazon.com/images/M/MV5BYTY2Zj...,https://www.imdb.com/title/tt1190634/?ref_=sr_t_1,The Boys,2019–,tv
1,https://m.media-amazon.com/images/M/MV5BM2QzMG...,https://www.imdb.com/title/tt11198330/?ref_=sr...,House of the Dragon,2022–,tv
2,https://m.media-amazon.com/images/M/MV5BZjE0Mj...,https://www.imdb.com/title/tt14452776/?ref_=sr...,The Bear,2022–,tv
3,https://m.media-amazon.com/images/M/MV5BY2RlMm...,https://www.imdb.com/title/tt12262202/?ref_=sr...,The Acolyte,2024–,tv
4,https://m.media-amazon.com/images/M/MV5BY2ZiOD...,https://www.imdb.com/title/tt8740790/?ref_=sr_t_5,Bridgerton,2020–,tv
5,https://m.media-amazon.com/images/M/MV5BN2IzYz...,https://www.imdb.com/title/tt0944947/?ref_=sr_t_6,Game of Thrones,2011–2019,tv
6,https://m.media-amazon.com/images/M/MV5BNTIxMm...,https://www.imdb.com/title/tt7440726/?ref_=sr_t_7,Your Honor,2020–2023,tv
7,https://m.media-amazon.com/images/M/MV5BZjkzMm...,https://www.imdb.com/title/tt0773262/?ref_=sr_t_8,Dexter,2006–2013,tv
8,https://m.media-amazon.com/images/M/MV5BNmEwYj...,https://www.imdb.com/title/tt11379456/?ref_=sr...,My Lady Jane,2024–,tv
9,https://m.media-amazon.com/images/M/MV5BZjVlNT...,https://www.imdb.com/title/tt31433814/?ref_=sr...,Doctor Who,2023–,tv


In [4]:
##########################
###     FUNCTIONS      ###
##########################

In [5]:
# [ input  ] words: string
# [ output ] string
# [  goal  ] replaces spaces with _
def remove_ws(words):
    no_ws = words.replace(" ", "_")
    return no_ws

In [6]:
# [ input  ] search_term: string
# [ output ] string
# [  goal  ] creates wikipedia link with the input string
def get_title_link(search_term):
    wiki_search = wikipedia.search(search_term)[0]
    wiki_search_no_ws = remove_ws(wiki_search)
    wiki_url = wiki_url_base + "/wiki/" + wiki_search_no_ws
    
    return wiki_url

In [98]:
# [ input  ] url: string
# [ output ] bs4 object
# [  goal  ] gets the soup for a url
def get_soup(url):
    data = requests.get(url, headers=headers).text
    soup = BeautifulSoup(data,'html.parser') # full page
    
    return soup

In [8]:
# [ input  ] soup: soup, parent: string, child: string
# [ output ] bs4 object
# [  goal  ] looks for element of child class in element of parent class
def find_child_in_soup(soup, parent, child):
    child = soup.find(f"{parent}",{"class":f"{child}"})
    
    return child

In [9]:
# [ input  ] soup: soup, infobox: bs4 object
# [ output ] array
# [  goal  ] determines if the relevant fields in a wikipedia infobox exist and if it does analyse infobox
def do_infobox_fields_exist(soup, infobox):
    infobox_fields = infobox.find_all('th', {'class' : 'infobox-label'})
    does_starring_field_exist = bool(re.search("Starring", str(infobox_fields)))
    does_voices_field_exist = bool(re.search("Voices of", str(infobox_fields)))
    
    if does_voices_field_exist is False and does_starring_field_exist is False:
        return [False, False]
    
    else :
        return field_results(soup, infobox, does_starring_field_exist, does_voices_field_exist)

In [10]:
# [ input  ] soup: soup, infobox: bs4, does_starring_field_exist: bool, does_voices_field_exist: bool
# [ output ] array
# [  goal  ] extracts cast list and cast links from wikipedia infobox if these exist
def field_results(soup, infobox, does_starring_field_exist, does_voices_field_exist):
    if does_voices_field_exist is True:
        th_pattern = voice_pattern
    elif does_starring_field_exist is True:
        th_pattern = starring_pattern
        
    th = soup.find('th', string=th_pattern).parent
    li_list = th.find_all('li')
    
    cast_list = []
    cast_links = []
    
    for li in li_list:
        a = li.find('a')
        if a == None :
            cast_list.append(li.text)
            cast_links.append("")
        else :
            cast_list.append(a.text)
            cast_links.append(a['href'])
            
    if len(cast_list) == 0 :
        return [False, False]
    else :
        return [cast_list, cast_links]

In [11]:
# [ input  ] title: string, year: string, tv_or_film: string
# [ output ] array
# [  goal  ] finds wikipedia link for a given title and then extracts cast list from page
def get_cast_from_wiki(title, year, tv_or_film): 
    
    search_term = f"{title} {year} {tv_or_film}"
    wiki_url = get_title_link(search_term)
    
    soup = get_soup(wiki_url)
    infobox = find_child_in_soup(soup, "table", "infobox vevent")
    
    if infobox is None :
        return [[],[]]
        
    else :        
        cast_list = []
        cast_list = do_infobox_fields_exist(soup, infobox)
        
        cast_names = cast_list[0]
        cast_links = cast_list[1]
        
        return cast_list

In [101]:
# [ input  ] title_id: string (imdb id)
# [ output ] array
# [  goal  ] extracts cast from imdb title page
def title_specs(title_id):
    
    url = imdb_base_url + title_id
    
    soup = get_soup(url)
    
    # CAST
    # grab top cast section of imdb page
    t = soup.find_all("div", {"class": "sc-bfec09a1-7 dpBDvu"})
    s = soup.find_all("a", {"class": "sc-bfec09a1-1 gCQkeh"})
    
    cast_list = []
    cast_links = []
    
    # get cast names
    for a in s:
        cast_list.append(str(a.string))
        cast_links.append("")
        
    return [cast_list, cast_links]

In [102]:
title_specs("tt0111161")

[['Tim Robbins',
  'Morgan Freeman',
  'Bob Gunton',
  'William Sadler',
  'Clancy Brown',
  'Gil Bellows',
  'Mark Rolston',
  'James Whitmore',
  'Jeffrey DeMunn',
  'Larry Brandenburg',
  'Neil Giuntoli',
  'Brian Libby',
  'David Proval',
  'Joseph Ragno',
  'Jude Ciccolella',
  'Paul McCrane',
  'Renee Blaine',
  'Scott Mann'],
 ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']]

In [13]:
# [ input  ] name: string
# [ output ] bool
# [  goal  ] checks if wikipedia page exists
def does_wiki_exist(name) :
    wiki_page = wiki_wiki.page(name.title())
    does_page_exist = wiki_page.exists()
    
    return does_page_exist

In [14]:
# [ input  ] url: string
# [ output ] string (img link)
# [  goal  ] extracts image from wikipedia infobox
def get_infobox_image(url):
    soup = get_soup(url)
    
    image = find_child_in_soup(soup, "a", "image")
    image_str = str(image)
    
    try:
        found = re.search('src="//(.+?)"', image_str).group(1)
    except AttributeError:
        found = ''
        
    return found

In [15]:
# [ input  ] url: string
# [ output ] subj_img: string, parent_data: object
# [  goal  ] extracts data from wikipedia infobox
def extract_infobox_data(url):
    soup = get_soup(url)
    
    bio_vcard = find_child_in_soup(soup, "table", "infobox biography vcard")
    vcard = find_child_in_soup(soup, "table", "infobox vcard")
    
    if bio_vcard is None and vcard is None : 
        return '', []
    else :
        if bio_vcard is not None :
            infobox = bio_vcard
        else :
            infobox = vcard
        
        blue_links = parse_infobox_parent_fields(soup, infobox)
        subj_img = get_infobox_image(url)
        
        parent_data = []
        
        if len(blue_links) != 0:
            for link in blue_links:
                name = link.partition('/wiki/')[2]
                name_wws = name.replace("_", " ")
                
                wiki_link = wiki_url_base + "/wiki/" + name
                image = get_infobox_image(wiki_link)
                
                parent = {
                    'name': name_wws,
                    'image': image,
                    'link': wiki_link
                }
                
                parent_data.append(parent)
        
        return subj_img, parent_data
    

In [16]:
# [ input  ] soup: soup, infobox: bs4
# [ output ] array
# [  goal  ] extracts parent / relative data from a wikipedia infobox
def parse_infobox_parent_fields(soup, infobox):
    
    infobox_fields = infobox.find_all('th', {'class' : 'infobox-label'})
    
    try :
        does_parent_field_exist = bool(re.search("Parent", str(infobox_fields)))
        does_relative_field_exist = bool(re.search("Relative", str(infobox_fields)))

        if does_parent_field_exist is False and does_relative_field_exist is False:
            return [] # parent field not listed in infobox -> not a nepo baby 

        else :
            if does_parent_field_exist is True : 
                return get_parent_links(soup)
            else :
                return get_rel_links(soup)
            
    except AttributeError:
        pass


In [17]:
# [ input  ] soup: soup
# [ output ] array
# [  goal  ] extracts parent data from a wikipedia infobox
def get_parent_links(soup) : 
    parent_field = soup.find('th', string=parent_pattern).parent
    parent_a_tags = parent_field.find_all('a')
                        
    if len(parent_a_tags) == 0 :
        return [] # parents listed in infobox but not linked -> not a nepo baby

    else : # nepo baby!
        parent_wiki_list = []

        for link in parent_field.find_all('a'):
            parent_wiki = link.get('href')
            parent_wiki_link = wiki_url_base + parent_wiki
            parent_wiki_list.append(parent_wiki_link)

            parent_wiki_list[:] = [x for x in parent_wiki_list if "cite_note" not in x] # cited entries are in <a href> tags so remove those links here

    return parent_wiki_list

In [18]:
# [ input  ] soup: soup
# [ output ] array
# [  goal  ] extracts relative data from a wikipedia infobox
def get_rel_links(soup) :
    relative_field = soup.find('th', string=relative_pattern).parent
    relative_td_tags = relative_field.find_all('td')

    list_of_relatives = re.split('</li>|<br/>', str(relative_td_tags))

    parent_wiki_list = []

    if len(relative_field.find_all('a')) == 0 :
        return []
    else :
        for i in range(len(list_of_relatives)):
            if bool(re.search(not_nepo_relationships, list_of_relatives[i])) is True:
                pass

            else :
                try :
                    href_match = re.search(r'href=\"(.*)\" title=', list_of_relatives[i])
                    parent_wiki = href_match.group(1)
                    parent_wiki_link = wiki_url_base + parent_wiki
                    parent_wiki_list.append(parent_wiki_link)

                except AttributeError :
                    pass  

        if len(parent_wiki_list) == 0 :
            return []
        else :
            return parent_wiki_list


In [19]:
# [ input  ] name: string, wiki_name_link: string
# [ output ] object
# [  goal  ] determines if name is a nepo babe based on their wikipedia page
def are_they_a_nepo(name, wiki_name_link = ""):
    
    poi = {'name': name,
          'image': '',
          'link': wiki_name_link,
          'nepo': False
          }
    
    if name is False or name == 'nan' :
        return poi
    
    if wiki_name_link != "" :
        url = wiki_url_base + wiki_name_link
        does_page_exist = True
    else :
        subject = remove_ws(name.title())
        url = wiki_url_base + "/wiki/" + subject
        print(url)
        does_page_exist = does_wiki_exist(name)
        
    if does_page_exist is False :
        poi['parents'] = []

    else : 
        subj_image, parent_data = extract_infobox_data(url)
        poi['link'] = url
        poi['image'] = subj_image
        poi['parents'] = parent_data
        
        if len(poi['parents']) != 0 :
            poi['nepo'] = True

    return poi